In [54]:
import sys
sys.path.insert(0, 'C:/Users/Santiago/Desktop/Facultad/Fisica/Simulaciones')
import ModSimPy as MSP
import numpy as np
import math
import matplotlib.pyplot as plt

In [55]:
%matplotlib

Using matplotlib backend: Qt5Agg


# Introduction

The aim of this notebook is to implent step by step the Vicsek Model refereced in ...(my git).

## ModSimPy documentation

This module contains the 'AnalysisVicsek' class and some subroutines, from which we will only use: 'Graficador' and 'ExtractTauEq'.

More detail will be given when required along the analysis. However, to begin with, lets comment on the initialization of the 
'AnalysisVicsek' class.

Firstly, one would run the Vicsek2D simulation, following the REEDME.txt file, which will generate the dessired outfiles.

Secondly, one initiates the AnalysisVicsek object as:
ObjV= MSP.AnalysisVicsek(run_number, inFiles, type_of_Analysis, parameters)

> - **run_number**: enumerates the object to keep track. Might be used as index.
> - **inFiles**: files to read data from
> - **type_of_Analysis**: either 'A' or 'V', again to keep track of the analysis
> - **parameters**: [N,v0,rho,MidoCada], basically the same parameters form the file 'Entrada_Parametros_Vicsek.dat'.

# Step 1: initialization

We will divide the objects into *Angular* and *Vectorial*

In [56]:
nroFilesA=0
nroFilesV=0

InFile=[]
AngInFiles=[]
VecInFiles=[]

### Append files to be red:

In [57]:
for j in range(10,15):
    namearch  = 'VAvsV_variandoN\Entrada_Parametros_Vicsek'+str(j)+'.dat'
    acorrFile='VAvsV_variandoN\Acorr_VA'+str(j)+'.dat'
    PhiFile='VAvsV_variandoN\CrudosVicsekA'+str(j)+'.dat'
    PhiSqrFile='VAvsV_variandoN\CrudosVicsekA'+str(j)+'_Sqr.dat'
    AngInFiles.append([acorrFile,PhiFile,PhiSqrFile])
    nroFilesA+=1
    InFile.append(namearch)

*Note 1.1*: the *Vectorial* analysis requires an extra parameter, G, so add it here.

*Note 1.2*: the number of analysis we will do for this type is going to be 3 instead of 5. We will take N = 225,625,1024.

In [58]:
G=0
decG = math.modf(G)[0]
for j in range(10,15,2):
    namearch  = 'VAvsV_variandoN\Entrada_Parametros_Vicsek'+str(j)+'.dat'
    acorrFile='VAvsV_variandoN\Acorr_VV'+str(j)+'_G'+str(int(G))+'p'+str(int(decG*10))+'.dat'
    PhiFile='VAvsV_variandoN\CrudosVicsekV'+str(j)+'_G'+str(int(G))+'p'+str(int(decG*10))+'.dat'
    PhiSqrFile='VAvsV_variandoN\CrudosVicsekvV'+str(j)+'_G'+str(int(G))+'p'+str(int(decG*10))+'.dat'
    VecInFiles.append([acorrFile,PhiFile,PhiSqrFile])
    nroFilesV+=1

### Get parameters
This are the same for both types of analysis.

In [59]:
N=[]
v0=[]
rho=[]
MidoCada=[]

parameters=[]
for j in range(nroFilesA):
    MSP.ReadParameters(InFile[j],N,v0,rho,MidoCada,True)    
    parameters.append([N[j],v0[j],rho[j],MidoCada[j]])

N = [225]
rho = [1.0]
$\Delta$ t = [200]
N = [225, 400]
rho = [1.0, 1.0]
$\Delta$ t = [200, 200]
N = [225, 400, 625]
rho = [1.0, 1.0, 1.0]
$\Delta$ t = [200, 200, 200]
N = [225, 400, 625, 841]
rho = [1.0, 1.0, 1.0, 1.0]
$\Delta$ t = [200, 200, 200, 200]
N = [225, 400, 625, 841, 1024]
rho = [1.0, 1.0, 1.0, 1.0, 1.0]
$\Delta$ t = [200, 200, 200, 200, 200]


'ReadParameters' from the module MSP: it reads the 'Entrada_Parametrs_Vicsek.dat' kind of files and returns the relevant parameters. If one replaces True for False, then it will not print the parameters.

For a more convenient use we store the in the parameters list.

### Initiate the objects

In [60]:
VA_OBJ=[]
VV_OBJ=[]

for j in range(nroFilesA):
    VA_OBJ.append(MSP.AnalysisVicsek(j,
                                     AngInFiles[j],
                                     'A',
                                     parameters[j]))
parametersV=[parameters[k] for k in range(0,len(parameters),2)]
# print(parametersV)
for j in range(nroFilesV):
    VV_OBJ.append(MSP.AnalysisVicsek(j,
                                     VecInFiles[j],
                                     'V',
                                     parametersV[j]))

*Note 1.3*: when we initialize the objects then we can get their attributes. The way this class does that is throw a method called:
'getVariables', which takes in as argument the files you want to read data from. It returns the *noise*, *time* and *X variable*.

Right away we can get the noise array, which we will call $\eta$. We only get it once, because it will be the same for all analysis.

In [61]:
eta=VA_OBJ[0].vicsekNoise

# Step 2: Autocorrelation study

### Get the Acor and time array for each noise.

In [62]:
tACTANG=[]
ACTANG=[]
for j in range(nroFilesA):
    tACTANG.append(VA_OBJ[j].acorrTime)
    ACTANG.append(VA_OBJ[j].acorr)
    
tACTVEC=[]
ACTVEC=[]
for j in range(nroFilesV):
    tACTVEC.append(VV_OBJ[j].acorrTime)
    ACTVEC.append(VV_OBJ[j].acorr)

### Plot Acorr as function of time for each $\eta$

In [63]:
labels = [r'$\eta = %s$' % eta[j] for j in range(len(eta))]
formatACT=['k.-','r.-','m.-','c.-','b.-','g.-','y.-','gv-','kv-','rv-']


In [64]:
graficoACT_A=[]
for k in range(nroFilesA):
    tituloAcorr=r'Anuglar - $N = %s$, $\rho = %s$' %(N[k],rho[k])
    graficoACT_A.append(
            [[tACTANG[k],ACTANG[k][j],None,formatACT[j],labels[j]]
            for j in range(0,len(eta),3)]
            )
    plt.figure(k)
    MSP.Graficador(graficoACT_A[k],'$t [MCS]$','$Acorr$',tituloAcorr)

In [65]:
graficoACT_V=[]
for k in range(nroFilesV):
    tituloAcorr='Vectorial - $N = %s$, $\rho = %s$'% (N[2*k],rho[k])
    graficoACT_V.append(
            [[tACTVEC[k],ACTVEC[k][j],None,formatACT[j],labels[j]]
            for j in range(len(eta))]
            )
    plt.figure(5+k)
    MSP.Graficador(graficoACT_V[k],'$t [MCS]$','$Acorr$',tituloAcorr)

We have done that just to check and get an idea of Acorr's behaviour.

### Extract the equilibrium time, known as $\tau_{eq}$
To do so, we need to define a fuction to fit the curves drawn in the previous cells. And we will need to provide a seed to get that fitting correctly.

In [66]:
def f(x,tau,b,y0): 
    return b*np.exp(-x/tau) + y0

In [67]:
# armar array con las semillas para cada archivo.
initGuessA=[[[MidoCada[k],ACTANG[k][j][0],0] for j in range(len(eta))]for k in range(nroFilesA)]#semilla para el ajuste  
# obtener el tiempo de eq. para cada curva de ACT
tauEq_A=[VA_OBJ[k].get_tauEq(f,initGuessA[k]) for k in range(nroFilesA)]

CAREFUL: FittingFunction must be like F(x,tau,A,y0,**p)
CAREFUL: FittingFunction must be like F(x,tau,A,y0,**p)


C:\Users\Santiago\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  
C:\Users\Santiago\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:779: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


CAREFUL: FittingFunction must be like F(x,tau,A,y0,**p)
CAREFUL: FittingFunction must be like F(x,tau,A,y0,**p)
CAREFUL: FittingFunction must be like F(x,tau,A,y0,**p)


In [68]:
# armar array con las semillas para cada archivo.
initGuessV=[[[MidoCada[k],ACTVEC[k][j][0],0] for j in range(len(eta))]for k in range(nroFilesV)]#semilla para el ajuste  
# obtener el tiempo de eq. para cada curva de ACT
tauEq_V=[VV_OBJ[k].get_tauEq(f,initGuessV[k]) for k in range(nroFilesV)]

CAREFUL: FittingFunction must be like F(x,tau,A,y0,**p)
CAREFUL: FittingFunction must be like F(x,tau,A,y0,**p)


C:\Users\Santiago\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  
C:\Users\Santiago\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:779: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


CAREFUL: FittingFunction must be like F(x,tau,A,y0,**p)


In [72]:
"""Visual result of $\tau_{eq}$"""

# [plt.plot(eta,tauEq_A[k],'o') for k in range(nroFilesA)]
# plt.show()

# [plt.plot(eta,tauEq_V[k],'o') for k in range(nroFilesV)]
# plt.show()


'Visual result of $\tau_{eq}$'

Now that we have the different $\tau$s we can calculate the mean value of the *Order Parameter* $|\phi|$

# Step 3: Get and plot relevant cuantities

## *Angular*

In [73]:
PhiMean_A=[VA_OBJ[k].get_OP_Mean(tauEq_A[k]) for k in range(nroFilesA)]
PhiVar_A=[VA_OBJ[k].get_OP_Var(tauEq_A[k]) for k in range(nroFilesA)]
PhiCB_A=[VA_OBJ[k]._calc_X_BinderCumulant(tauEq_A[k]) for k in range(nroFilesA)]

In [74]:
StatsLabels_A=[r'Angular - $N=%s$'%N[k] for k in range(nroFilesA)]
formatos=['yo--','go--','bo--','ro--','mo--']

In [75]:
graficoOP=[[eta,PhiMean_A[k],None,formatos[k],StatsLabels_A[k]] for k in range(nroFilesA)]
graficoSuc=[[eta,PhiVar_A[k],None,formatos[k],StatsLabels_A[k]]for k in range(nroFilesA)]
graficosCB=[[eta[:],PhiCB_A[k][:],None,formatos[k],StatsLabels_A[k]]for k in range(nroFilesA)]

In [76]:
plt.figure(9)
MSP.Graficador(graficoOP,'$\eta$','<|$\phi$|>','Limite termodinámico, Angular $vs$ Vectorial')
plt.figure(10)
MSP.Graficador(graficoSuc,'$\eta$','Suceptibilidad','Limite termodinámico, Angular $vs$ Vectorial')
plt.figure(11)
MSP.Graficador(graficosCB,'$\eta$','CumulanteBinder','Limite termodinámico, Angular $vs$ Vectorial')

## *Vectorial*

In [77]:
PhiMean_V=[VV_OBJ[k].get_OP_Mean(tauEq_V[k]) for k in range(nroFilesV)]
PhiVar_V=[VV_OBJ[k].get_OP_Var(tauEq_V[k]) for k in range(nroFilesV)]
PhiCB_V=[VV_OBJ[k]._calc_X_BinderCumulant(tauEq_V[k]) for k in range(nroFilesV)]

In [78]:
StatsLabels_V=[r'Vectorial - $N=%s$'%N[2*k] for k in range(nroFilesV)]
formatosV=['y*--','g*--','r*--','m*--','b*--',]

In [79]:
graficoOP_V=[[eta,PhiMean_V[k],None,formatosV[k],StatsLabels_V[k]] for k in range(nroFilesV)]
graficoSuc_V=[[eta,PhiVar_V[k],None,formatosV[k],StatsLabels_V[k]]for k in range(nroFilesV)]
graficosCB_V=[[eta[:],PhiCB_V[k][:],None,formatosV[k],StatsLabels_V[k]]for k in range(nroFilesV)]

In [80]:
plt.figure(9)
MSP.Graficador(graficoOP_V,'$\eta$','<|$\phi$|>','Limite termodinámico, Angular $vs$ Vectorial')
plt.figure(12)
MSP.Graficador(graficoSuc_V,'$\eta$','Suceptibilidad','Limite termodinámico, Angular $vs$ Vectorial')
plt.figure(13)
MSP.Graficador(graficosCB_V,'$\eta$','CumulanteBinder','Limite termodinámico, Angular $vs$ Vectorial')